# Amazon Product Review #
# Importing Packages 

In [ ]:
# Importing Packages 
# These modules is used to data set loding and reading. It consists of (.Json, .csv) files. 
%matplotlib inline
import sqlite3
import json
import csv
print("************************Done*********************")

In [ ]:
# Importing Packages 
# These modules are used to preprocess data, manipulate data, marge and separate data.import pandas as pd
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import re
print("************************Done*********************")

In [ ]:
# Importing packages 
# This modules are used to cluster the data, extracting features, Converting word to vector, Classification and perfomance evaluation.
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.pipeline import FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from wordcloud import WordCloud, STOPWORDS
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
print("*********************Done*************************")

# Reading data from JSON file 

In [ ]:
df = pd.read_json('reviews_Office_Products.json',lines='True' )
df

In [ ]:
helpfulness = df[['helpful','asin']]
helpfulness

In [ ]:
# Split the helpful data frame to  helpful, total. It means Helpfulnes of the product eg: 2/3
df2 = pd.DataFrame(helpfulness)
df2[['helpful','total']] = pd.DataFrame(df2.helpful.values.tolist(), index=df2.index)
dfhelpful = df2[['asin', 'helpful','total']]
dfhelpful

In [ ]:
#Join total, helpful data frames to the main data frame. 
dfNew = df.drop('helpful',1)
frames = [dfNew, dfhelpful]
result = dfNew.join(dfhelpful[['helpful','total']])
result

# Create .csv file to store all modified data frame reviews. 

In [ ]:
result.to_csv('reviews.csv', sep=',', header=False,index=False)
result

# Load .csv data to sqlite for Fetch the score and recommendation summary using SQL queries.


In [ ]:
class csvrd(object):
    def csvFile(self):
        self.readFile('reviews.csv')
    def readFile(self, filename):
        conn = sqlite3.connect('amazonReviews.db')
        cur = conn.cursor() 
        cur.execute("""CREATE TABLE IF NOT EXISTS amazonReviews(asin INT,overall INT,reviewText varchar,reviewTime INTEGER, reviewerID varchar,reviewerName varchar,summary varchar,unixReviewTime INTEGER,helpful INT,total INT)""")
        filename.encode('utf-8')
        print ("Amazon Reviews table executed in DB ")
        with open(filename) as f:
            reader = csv.reader(f)
            for field in reader:
                cur.execute("INSERT INTO amazonReviews VALUES (?,?,?,?,?,?,?,?,?,?);", field)

        print ("**************CSV Loaded into SQLite******************")
        conn.commit()
        conn.close()

c = csvrd().csvFile()

# Reading the data from SQLite


In [ ]:
con = sqlite3.connect('amazonReviews.db')
pd.read_sql_query("SELECT * FROM amazonReviews LIMIT 20", con)

In [ ]:
# ignoring the reviews having overall score equal to 3. 
#If the overall score is above 3, then the label for it will be set to Positive
#else it will be set to Negative
reviews = pd.read_sql_query("""
SELECT 
  overall, 
  summary, 
  helpful, 
  total
FROM amazonReviews 
WHERE overall != 3""", con)
#display data
reviews.head(5)

In [ ]:
##sentiment column depicts the numeric score of being positive or negative
##usefulScore column depicts the boolean value of total number of votes
reviews["sentiment"] = reviews["overall"].apply(lambda score: "positive" if score > 3 else "negative")
reviews["usefulScore"] = (reviews["helpful"]/reviews["total"]).apply(lambda n: "useful" if n > 0.8 else "useless")
reviews.head(5)

In [ ]:
# Reviews over all score is equal to 5
reviews[reviews.overall == 5].head(5)


In [ ]:
# #Rows having overall score equal to 1
reviews[reviews.overall == 1].head(5)


# # Feature extraction from collected reviews text 

In [ ]:
# Assign new dimension to each word and give the word counts
regEx = re.compile('[^a-z]+')
def cleanReviews(reviewText):
    reviewText = reviewText.lower()
    reviewText = regEx.sub(' ', reviewText).strip()
    print(reviewText)
    return reviewText
reviews["summaryClean"] = reviews["summary"].apply(cleanReviews)

train, test = train_test_split(reviews.sample(n=150000, random_state=1), test_size=0.2)
print("%d items in training data, %d in test data" % (len(train), len(test)))


In [ ]:
### Using TfidfTransformer().fit_transofrm 
##to fit the train and test data
countVector = CountVectorizer(min_df = 1, ngram_range = (1, 4))
X_train_counts = countVector.fit_transform(train["summaryClean"])



#Applying tfidf to term frequency
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

X_new_counts = countVector.transform(test["summaryClean"])
X_test_tfidf = tfidf_transformer.transform(X_new_counts)

y_train = train["sentiment"]
y_test = test["sentiment"]

prediction = dict()

# Hybrid clustering

In [ ]:
cluster = df.groupby("overall")["summary"].apply(list)
cluster = pd.DataFrame(cluster)
cluster.to_csv("cluster.csv")
#writing cluster data into .csv file. 
cluster1 = pd.read_csv("cluster.csv")
cluster1["summaryClean"] = cluster1["summary"].apply(cleanReviews)


In [ ]:
## stop words cleaning 
stopwords = set(STOPWORDS)

#mpl.rcParams['figure.figsize']=(8.0,6.0)    #(6.0,4.0)
mpl.rcParams['font.size']=12                #10 
mpl.rcParams['savefig.dpi']=100             #72 
mpl.rcParams['figure.subplot.bottom']=.1 

def show_wordcloud(data, title = None):
    wordcloud = WordCloud(
        background_color='white',
        stopwords=stopwords,
        max_words=200,
        max_font_size=40, 
        scale=3,
        random_state=1 # chosen at random by flipping a coin; it was heads
    ).generate(str(data))
    
    fig = plt.figure(1, figsize=(8, 8))
    plt.axis('off')
    if title: 
        fig.suptitle(title, fontsize=20)
        fig.subplots_adjust(top=2.3)

    plt.imshow(wordcloud)
    plt.show()
    
show_wordcloud(reviews["summaryClean"])

In [ ]:
show_wordcloud(reviews[reviews.overall == 1]["summaryClean"], title = "Low scored words")

In [ ]:
show_wordcloud(reviews[reviews.overall == 5]["summaryClean"], title = "High scored words")

In [ ]:
show_wordcloud(reviews[reviews.overall == 2]["summaryClean"], title = "Average scored words")

# Naive Bayes with Multinomial

In [ ]:
model = MultinomialNB().fit(X_train_tfidf, y_train)
prediction['Multinomial'] = model.predict(X_test_tfidf)

# Naive Bayes Multinomial

In [ ]:
model = BernoulliNB().fit(X_train_tfidf, y_train)
prediction['Bernoulli'] = model.predict(X_test_tfidf)

# Logistic regression 

In [ ]:
logreg = LogisticRegression(C=1e5)
logreg_result = logreg.fit(X_train_tfidf, y_train)
prediction['Logistic'] = logreg.predict(X_test_tfidf)

## SVM 

In [ ]:

svclassifier = SVC(kernel='linear')
svclassifier.fit(X_train_tfidf, y_train)
prediction['SVM'] = svclassifier.predict(X_test_tfidf)
#classifier = KNeighborsClassifier(n_neighbors=5,n_jobs=5)
#classifier.fit(X_train_tfidf, y_train)
# Import the model we are using
#from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
#rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
#rf.fit(X_train_tfidf, y_train);

## KNN

In [ ]:
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train_tfidf, y_train)
prediction['KNN'] = classifier.predict(X_test_tfidf)


## Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train_tfidf,y_train)

#Predict the response for test dataset
prediction['DecisionTree'] = clf.predict(X_test_tfidf)



## SVM accuracy

In [ ]:
accuracy_score(y_test, prediction['SVM'])

## KNN accuracy

In [ ]:
accuracy_score(y_test, prediction['KNN'])

## Logistic Regression accuracy 

In [ ]:
accuracy_score(y_test, prediction['Logistic'])

## Naive Bayes Multinomial accuracy 

In [ ]:
accuracy_score(y_test, prediction['Multinomial'])

## Naive Bayes Bernoulli accuracy 

In [ ]:
accuracy_score(y_test, prediction['Bernoulli'])

## Decision Tree accuracy

In [ ]:
accuracy_score(y_test, prediction['DecisionTree'])

## AUC-ROC

In [ ]:
def formatt(x):
    if x == 'negative':
        return 0
    return 1
vfunc = np.vectorize(formatt)

cmp = 0
colors = ['b', 'g', 'y', 'm', 'k','r']
plt.figure(figsize=(20,10))
for model, predicted in prediction.items():
    false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test.map(formatt), vfunc(predicted))
    roc_auc = auc(false_positive_rate, true_positive_rate)
    plt.plot(false_positive_rate, true_positive_rate, colors[cmp], label='%s: AUC %0.2f'% (model,roc_auc))
    cmp += 1


plt.title('Classifiers comparaison with ROC')
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([-0.1,1.2])
plt.ylim([-0.1,1.2])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

# Visualize the accuracy, recall and f1-score for Logistic Regression

In [ ]:
print(metrics.classification_report(y_test, prediction['Logistic'], target_names = ["positive", "negative"]))

In [ ]:
accuracy_score(y_test, prediction['Logistic'])

In [ ]:
def plot_confusion_matrix(matrix, title='Confusion matrix', cmap=plt.cm.Blues, labels=["positive", "negative"]):
    plt.imshow(matrix, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(labels))
    plt.xticks(tick_marks, labels, rotation=45)
    plt.yticks(tick_marks, labels)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
# Compute confusion matrix
matrix = confusion_matrix(y_test, prediction['Logistic'])
np.set_printoptions(precision=2)
plt.figure()
plot_confusion_matrix(matrix)    

matrix_normalized = matrix.astype('float') / matrix.sum(axis=1)[:, np.newaxis]
plt.figure()
plot_confusion_matrix(matrix_normalized, title='Normalized confusion matrix')
plt.show()

# Visualize the accuracy, recall and f1-score for Naive Bayes Berrnoulli

In [ ]:
print(metrics.classification_report(y_test, prediction['Bernoulli'], target_names = ["positive", "negative"]))

In [ ]:
accuracy_score(y_test, prediction['Bernoulli'])

In [ ]:
def plot_confusion_matrix(matrix, title='Confusion matrix', cmap=plt.cm.Blues, labels=["positive", "negative"]):
    plt.imshow(matrix, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(labels))
    plt.xticks(tick_marks, labels, rotation=45)
    plt.yticks(tick_marks, labels)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
# Compute confusion matrix
matrix = confusion_matrix(y_test, prediction['Bernoulli'])
np.set_printoptions(precision=2)
plt.figure()
plot_confusion_matrix(matrix)    

matrix_normalized = matrix.astype('float') / matrix.sum(axis=1)[:, np.newaxis]
plt.figure()
plot_confusion_matrix(matrix_normalized, title='Normalized confusion matrix')
plt.show()

# Visualize the accuracy, recall and f1-score for Naive Bayes Multinomial

In [ ]:
print(metrics.classification_report(y_test, prediction['Multinomial'], target_names = ["positive", "negative"]))

In [ ]:
accuracy_score(y_test, prediction['Multinomial'])

In [ ]:
def plot_confusion_matrix(matrix, title='Confusion matrix', cmap=plt.cm.Blues, labels=["positive", "negative"]):
    plt.imshow(matrix, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(labels))
    plt.xticks(tick_marks, labels, rotation=45)
    plt.yticks(tick_marks, labels)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
# Compute confusion matrix
matrix = confusion_matrix(y_test, prediction['Multinomial'])
np.set_printoptions(precision=2)
plt.figure()
plot_confusion_matrix(matrix)    

matrix_normalized = matrix.astype('float') / matrix.sum(axis=1)[:, np.newaxis]
plt.figure()
plot_confusion_matrix(matrix_normalized, title='Normalized confusion matrix')
plt.show()

# Visualize the accuracy, recall and f1-score for SVM

In [ ]:
print(metrics.classification_report(y_test, prediction['SVM'], target_names = ["positive", "negative"]))

In [ ]:
accuracy_score(y_test, prediction['SVM'])

In [ ]:
def plot_confusion_matrix(matrix, title='Confusion matrix', cmap=plt.cm.Blues, labels=["positive", "negative"]):
    plt.imshow(matrix, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(labels))
    plt.xticks(tick_marks, labels, rotation=45)
    plt.yticks(tick_marks, labels)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
# Compute confusion matrix
matrix = confusion_matrix(y_test, prediction['SVM'])
np.set_printoptions(precision=2)
plt.figure()
plot_confusion_matrix(matrix)    

matrix_normalized = matrix.astype('float') / matrix.sum(axis=1)[:, np.newaxis]
plt.figure()
plot_confusion_matrix(matrix_normalized, title='Normalized confusion matrix')
plt.show()

# Visualize the accuracy, recall and f1-score for KNN

In [ ]:
print(metrics.classification_report(y_test, prediction['KNN'], target_names = ["positive", "negative"]))

In [ ]:
accuracy_score(y_test, prediction['KNN'])

In [ ]:
def plot_confusion_matrix(matrix, title='Confusion matrix', cmap=plt.cm.Blues, labels=["positive", "negative"]):
    plt.imshow(matrix, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(labels))
    plt.xticks(tick_marks, labels, rotation=45)
    plt.yticks(tick_marks, labels)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
# Compute confusion matrix
matrix = confusion_matrix(y_test, prediction['KNN'])
np.set_printoptions(precision=2)
plt.figure()
plot_confusion_matrix(matrix)    

matrix_normalized = matrix.astype('float') / matrix.sum(axis=1)[:, np.newaxis]
plt.figure()
plot_confusion_matrix(matrix_normalized, title='Normalized confusion matrix')
plt.show()

# Visualize the accuracy, recall and f1-score for DecisionTree

In [ ]:
print(metrics.classification_report(y_test, prediction['DecisionTree'], target_names = ["positive", "negative"]))

In [ ]:
accuracy_score(y_test, prediction['DecisionTree'])

In [ ]:
def plot_confusion_matrix(matrix, title='Confusion matrix', cmap=plt.cm.Blues, labels=["positive", "negative"]):
    plt.imshow(matrix, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(labels))
    plt.xticks(tick_marks, labels, rotation=45)
    plt.yticks(tick_marks, labels)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
# Compute confusion matrix
matrix = confusion_matrix(y_test, prediction['DecisionTree'])
np.set_printoptions(precision=2)
plt.figure()
plot_confusion_matrix(matrix)    

matrix_normalized = matrix.astype('float') / matrix.sum(axis=1)[:, np.newaxis]
plt.figure()
plot_confusion_matrix(matrix_normalized, title='Normalized confusion matrix')
plt.show()

# Displaying words depicting the best and worst features


In [ ]:
features = countVector.get_feature_names()
feature_coefs = pd.DataFrame(
    data = list(zip(features, logreg_result.coef_[0])),
    columns = ['feature', 'coefficient'])

feature_coefs.sort_values(by='coefficient')

# Testing the sentiments of few reviews 

In [ ]:
def testSentiments(model, testData):
    testCounts = countVector.transform([testData])
    testTfidf = tfidf_transformer.transform(testCounts)
    result = model.predict(testTfidf)[0]
    probability = model.predict_proba(testTfidf)[0]
    print("Sample estimated as %s: negative prob %f, positive prob %f" % (result.upper(), probability[0], probability[1]))

In [ ]:
testing_review=testSentiments(logreg, input('Enter Product review:'))
testing_review=testSentiments(logreg, input('Enter Product review:'))
testing_review=testSentiments(logreg, input('Enter Product review:'))
print('****************************END*******************************')